# CS105 Final Project

In [72]:
import pandas as pd
import numpy as np

Read Data In.

In [73]:
df = pd.read_csv('jeopardy.csv')
df.head(25)

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams
5,4680,2004-12-31,Jeopardy!,3-LETTER WORDS,$200,"In the title of an Aesop fable, this insect sh...",the ant
6,4680,2004-12-31,Jeopardy!,HISTORY,$400,Built in 312 B.C. to link Rome & the South of ...,the Appian Way
7,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$400,"No. 8: 30 steals for the Birmingham Barons; 2,...",Michael Jordan
8,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$400,"In the winter of 1971-72, a record 1,122 inche...",Washington
9,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$400,This housewares store was named for the packag...,Crate & Barrel


Specify subsets, here we're only going to look at these specific values.

In [74]:
df = df[['Round', 'Value', 'Question', 'Answer']]
df.head(5)

,Round,Value,Question,Answer
0,Jeopardy!,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,Jeopardy!,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,Jeopardy!,$200,The city of Yuma in this state has a record av...,Arizona
3,Jeopardy!,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,Jeopardy!,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


Clean Data to remove Instances of Final Jeopary and 'None' Values, as they do not factor into our calculations.

In [75]:
df = df[~df.Round.str.contains('Final')]
df = df[~df.Value.str.contains('None')]

Create new columns for both `Words in Question` and `Words in Answer` by spliting strings in cells by the amount of spaces.  This creates a list with all words in a cell, then counting the size of that list with the `len()` function gives us the amount of words in each cell.

In [76]:
df['Words in Question'] = df['Question'].str.split().str.len()
df['Words in Answer'] = df['Answer'].str.split().str.len()

Notice that Value is currently being treated as categorical variable. Clean up this column and cast them to a float because we'll need to perform mathematical operations on these columns.

In [77]:
df['Value'] = df['Value'].str.replace('$','').str.replace(',', '').astype(float)
df

C:\Users\chai\AppData\Local\Temp\ipykernel_30368\2060277076.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Value'] = df['Value'].str.replace('$','').str.replace(',', '').astype(float)


,Round,Value,Question,Answer,Words in Question,Words in Answer
0,Jeopardy!,200.0,"For the last 8 years of his life, Galileo was ...",Copernicus,18,1.0
1,Jeopardy!,200.0,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,19,2.0
2,Jeopardy!,200.0,The city of Yuma in this state has a record av...,Arizona,18,1.0
3,Jeopardy!,200.0,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,14,1.0
4,Jeopardy!,200.0,"Signer of the Dec. of Indep., framer of the Co...",John Adams,18,2.0
...,...,...,...,...,...,...
216924,Double Jeopardy!,2000.0,In 2006 the cast of this long-running hit emba...,Stomp,22,1.0
216925,Double Jeopardy!,2000.0,This Puccini opera turns on the solution to 3 ...,Turandot,14,1.0
216926,Double Jeopardy!,2000.0,In North America this term is properly applied...,a titmouse,18,2.0
216927,Double Jeopardy!,2000.0,"In Penny Lane, where this ""Hellraiser"" grew up...",Clive Barker,16,2.0
